## 18단계: 메모리 절약 모드

> 이번 단계에서는 DeZero의 메모리 사용을 개선할 수 있는 구조 두 가지를 도입합니다. \
첫 번째는 역전파 시 사용하는 메모리양을 줄이는 방법으로, 불필요한 미분 결과를 보관하지 않고 즉시 삭제합니다. \
두 번째는 '역전파가 필요 없는 경우용 모드'를 제공하는 것입니다. 이 모드에서는 불필요한 미분 계산을 생략합니다.

### 18.1 필요 없는 미분값 삭제

현재의 DeZero는 모든 변수가 미분값을 변수에 저장해두고 있다. 하지만 보통 계산을 위한 임시 변수들의 미분값은 불필요하기 때문에, 중간 변수에 대해 미분값을 제거해줄 수 있는 모드를 추가한다.


In [ ]:
import weakref
from heapq import heappush, heappop
import numpy as np


def as_array(x):
    if np.isscalar(x):
        return np.asarray(x)
    return x


class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]
        
        self.generation = max([x.generation for x in inputs])
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = [weakref.proxy(output) for output in outputs]
        
        return outputs if len(outputs) > 1 else outputs[0]

    def __lt__(self, other):
        return self.generation > other.generation
    
    def forward(self, xs):
        raise NotImplementedError()
    
    def backward(self, gys):
        raise NotImplementedError()



class Variable:
    def __init__(self, data):
        if not isinstance(data, (type(None), np.ndarray)):
            raise TypeError(f'{type(data)}은(는) 지원하지 않습니다.')
        
        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1
    
    # 미분값 유지 여부 옵션 추가
    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        
        funcs_heap = []
        seen_set = set()
        
        def add_func(f):
            if f not in seen_set:
                heappush(funcs_heap, f)
                seen_set.add(f)
        
        add_func(self.creator)
        
        while funcs_heap:
            f = heappop(funcs_heap)
            gys = [output.grad for output in f.outputs]
            
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)
            
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                
                if x.creator is not None:
                    add_func(x.creator)
            
            # retain_grad == False이면 중간 변수의 미분값을 모두 삭제
            if not retain_grad:
                for y in f.outputs:
                    # y().grad = None  # y는 약한 참조(weakref)
                    y.grad = None  # weakref.proxy 적용 버전
    
    def cleargrad(self):
        self.grad = None


class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
    
    def backward(self, gy):
        return gy, gy


class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx


def add(x0, x1):
    return Add()(x0, x1)

def square(x):
    return Square()(x)


x0 = Variable(np.array(1.0))
x1 = Variable(np.array(1.0))
t = add(x0, x1)
y = add(x0, t)
y.backward()

print(y.grad, t.grad)  # 중간 변수 미분값은 삭제됨
print(x0.grad, x1.grad)  # 말단의 미분값은 유지

None None
2.0 1.0


### 18.2 Function 클래스 복습

`Function` 클래스는 입력값들을 `self.inputs`라는 인스턴스 변수로 참조하고, 따라서 입력값의 참조 카운트를 증가시켜 변수들을 메모리에 유지시킨다. 이는 미분값을 계산할 때 필요하기 때문인데, 만약 미분값이 필요 없다면 (모델의 '추론' 과정 등) 오히려 메모리에서 회수하는게 나을 것이다.

### 18.3 Config 클래스를 활용한 모드 전환

DeZero에 '역전파 활성 모드'와 '역전파 비활성 모드'를 전환하는 구조를 추가한다.

In [16]:
class Config:
    enable_backprop = True  # True이면 역전파 활성 모드


class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]
        
        # 역전파 활성 모드 확인
        if Config.enable_backprop:
            self.generation = max([x.generation for x in inputs])  # Todo: __lt__와 충돌 가능
            for output in outputs:
                output.set_creator(self)
            self.inputs = inputs
            self.outputs = [weakref.proxy(output) for output in outputs]
        
        return outputs if len(outputs) > 1 else outputs[0]

    def __lt__(self, other):
        return self.generation > other.generation
    
    def forward(self, xs):
        raise NotImplementedError()
    
    def backward(self, gys):
        raise NotImplementedError()

In [21]:
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
    
    def backward(self, gy):
        return gy, gy


class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx


def add(x0, x1):
    return Add()(x0, x1)

def square(x):
    return Square()(x)

### 18.4 모드 전환

모드 전환 예시

In [24]:
# 역전파 활성 모드
Config.enable_backprop = True
x = Variable(np.ones((100, 100, 100)))
y = square(square(square(x)))
y.backward()
# -> 중간 변수들이 유지되고 역전파가 가능함

In [25]:
# 역전파 비활성 모드
Config.enable_backprop = False
x = Variable(np.ones((100, 100, 100)))
y = square(square(square(x)))
# -> 중간 변수들이 삭제되고 역전파가 불가능함
y.backward()  # 오류

AttributeError: 'NoneType' object has no attribute 'outputs'

### 18.5 with 문을 활용한 모드 전환

파이썬에는 `with`문을 활용한 '컨텍스트 전환' 기능이 있다. 대표적으로 파일의 `open`과 `close`에 사용된다.

In [26]:
# 파일 수정 예시
f = open('sample.txt', 'w')
f.write('hello world!')
f.close()  # 파일을 명시적으로 닫아줘야함
# 파일을 닫지 않으면 자원도 낭비되고, 다른 프로세스가 같은 파일을 참고할 때 race condition도 발생 가능함
# 또한 파일이 제대로 저장되지 않을 수도 있음 (파일 저장 버퍼 관련 문제)

# with문 예시
with open('sample.txt', 'w') as f:
    f.write('hello world!')
# with문을 빠져나오면 자동으로 f.close 메서드가 호출됨

이를 활용하여 with문 안에 들어오면 역전파 비활성 모드, with문을 나오면 활성 모드로 전환해주는 함수를 만들어보자. 파이썬 내장 모듈인 `contextlib` 모듈을 이용하면 쉽게 구현이 가능하다.

In [ ]:
# contextlib 모듈 사용 테스트

import contextlib

@contextlib.contextmanager
def config_test():
    print('start')  # 전처리
    # 오류 발생을 대비하여 try-finally 구문으로 감싸준다.
    try:
        yield
    finally:
        print('done')  # 후처리

with config_test():
    print('process...')

start
process...
done


In [ ]:
# Config 값 임시 수정용 contextlib 코드

import contextlib

@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)

Config.enable_backprop = True
print('with문 이전:', Config.enable_backprop)

with using_config('enable_backprop', False):
    print('with문 내부:', Config.enable_backprop)
    x = Variable(np.array(2.0))
    y = square(x)

print('with문 이후:', Config.enable_backprop)

with문 이전: True
with문 내부: False
with문 이후: True


In [ ]:
# with문 활용한 역전파 임시 비활성 함수 구현

def no_grad():
    return using_config('enable_backprop', False)

Config.enable_backprop = True
print('with문 이전:', Config.enable_backprop)

with no_grad():
    print('with문 내부:', Config.enable_backprop)
    x = Variable(np.array(2.0))
    y = square(x)

print('with문 이후:', Config.enable_backprop)

with문 이전: True
with문 내부: False
with문 이후: True


이상으로 메모리 효율적인 DeZero 구현을 마친다.